In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
from src.data_loader import read_data

train_data = read_data('train')
val_data = read_data('val')
test_data = read_data('test')

100%|██████████| 200/200 [00:00<00:00, 4177.09it/s]


In [3]:
import clip.clip as clip 

CLIP_MODEL,  clip_img_processor =clip.load('ViT-B/32', jit=False)

In [4]:
from src.config import BATCH_SIZE
import random 
import numpy as np 
import torch 

from src.networks.CLIP_distilBERT_unfreeze import CLIP_dBERT_unfreezed_layers_dataset

def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)
g = torch.Generator()
g.manual_seed(0)

def get_dataloader(data):
    dataset = CLIP_dBERT_unfreezed_layers_dataset(data, clip_img_processor)
    data_loader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE,shuffle=True, worker_init_fn=seed_worker, generator=g)
    return data_loader
    
train_loader = get_dataloader(train_data)
val_loader = get_dataloader(val_data)
test_loader = get_dataloader(test_data)

CLIP preprocessing: 100%|██████████| 200/200 [00:03<00:00, 63.54it/s]


In [5]:
from src.trainer import Trainer
from src.networks.CLIP_distilBERT_unfreeze import CLIP_dBERT_unfreezed_layers_Input_transformer, CLIP_dBERT_unfreezed_layers_Model
import torch.nn as nn
import torch 

trainer = Trainer(12)

#Set data
trainer.set_data(train_loader, val_loader)

#set model 
model = CLIP_dBERT_unfreezed_layers_Model(CLIP_MODEL).cuda()
input_transformer = CLIP_dBERT_unfreezed_layers_Input_transformer()
trainer.set_model(model, input_transformer)

#Backpropagation
loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=5e-4)
trainer.set_optimizer(optimizer, loss_fn)
trainer.train(l2_r=1e-3)
trainer.plot()

ImportError: cannot import name 'CLIP_dBERT_unfreezed_layers_transformer' from 'src.networks.CLIP_distilBERT_unfreeze' (c:\Users\ADITYA RATHORE\Desktop\CV_Project\src\networks\CLIP_distilBERT_unfreeze.py)

In [ ]:
trainer.test(test_loader)